   # Healthcare Quality Prediction

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import ElasticNet
import mlflow
import mlflow.sklearn

In [ ]:
#mlflow.set_tracking_uri("sqlite:///healthcare-quality.sqlite")

In [ ]:
mlflow.get_tracking_uri()

In [ ]:
# Loading data from a CSV file
df = pd.read_csv('./healthcare-quality.csv')

# Separating the target class ('quality') from remainder of the training data
X = df.drop(columns = 'quality')

y = df[['quality']]

# Splitting the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = 42)

## Model Training and Logging

In [ ]:
# Fix Conda Python version used by Seldon Core MLflow in Kubernetes
conda_env = mlflow.sklearn.get_default_conda_env()

for i, dep in enumerate(conda_env['dependencies']):
    if type(dep) == str and dep.startswith("python="):
        conda_env['dependencies'][i] = "python"

conda_env

In [ ]:
# Defining model parameters
for name, alpha, l1_ratio in [("healthcare-quality-model-a", 0.5, 0.5), ("healthcare-quality-model-b", 1.0, 0.5)]:
    # Running MLFlow script
    with mlflow.start_run():

        # Instantiating model with model parameters
        model = ElasticNet(alpha = alpha, l1_ratio = l1_ratio)

        # Fitting training data to the model
        model.fit(X_train, y_train)

        # Running prediction on validation dataset
        preds = model.predict(X_val)

        # Getting metrics on the validation dataset
        rmse = mean_squared_error(preds, y_val)
        abs_error = mean_absolute_error(preds, y_val)
        r2 = r2_score(preds, y_val)

        # Logging params and metrics to MLFlow
        mlflow.log_param('alpha', alpha)
        mlflow.log_param('l1_ratio', l1_ratio)
        mlflow.log_metric('rmse', rmse)
        mlflow.log_metric('abs_error', abs_error)
        mlflow.log_metric('r2', r2)

        # Logging model to MLFlow. Requires MLflow (Server) with DB backend
        mlflow.sklearn.log_model(
            sk_model = model,
            artifact_path = "",
            registered_model_name = name,
            conda_env = conda_env
        )

In [ ]:
df = mlflow.search_runs(filter_string="metrics.rmse < 1")

In [ ]:
df

In [ ]:
#Fetching Run ID for best model
# run_id = df.loc[df['metrics.rmse'].idxmin()]['run_id']
#best_model_url = df.loc[df['metrics.rmse'].idxmin()]["artifact_uri"]
# Load model as a PyFuncModel.
#best_model = mlflow.pyfunc.load_model(best_model_url)
#y_pred = best_model.predict(X_val)
#print(y_pred[0:10])

# Kubernetes, MLFlow, Seldon: ML Infrastructure as code

Infrastrucure as code in notebooks gives data scientists full flexibility to leverage the underlying automation to scale and deploy, training more models in parallel to get to prod faster. The MLFlow service will be accessed outside the cluster via network mesh overlay, and both models promoted and abutomatically load balanced by Seldon. Which will in turn auto-generate model API routes in Swagger:

 - Deploy AB model via Seldon Core to a local Kubernetes cluster. 
 - Generate some traffic to REST endpoint that is auto-generated. 
 - Optional: Remove AB models and all its pods and services from the cluster

## Deploy AB Model from MLflow to Seldon Core

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

model_a_version = client.get_latest_versions("healthcare-quality-model-a", stages=["None"])[0].version
model_b_version = client.get_latest_versions("healthcare-quality-model-b", stages=["None"])[0].version

In [ ]:
import seldon_core_utils

In [ ]:
seldon_core_utils.ab_deployment(
    name="healthcare_metrics",
    namespace="default",
    secret_name="mlflow-seldon-secret",
    model_a_name="healthcare-quality-model-a",
    model_a_version=model_a_version,
    model_b_name="healthcare-quality-model-b",
    model_b_version=model_b_version,
    model_a_traffic=50,
)

## Generate Traffic to AB Model

Wait for the model to be fully deployed, then execute the below statements!

In [ ]:
import requests
import time

headers = {"Content-Type": "application/json"}
body = {"data":{"ndarray":[[7,0.27,0.36,20.7,0.045,45,170,1.001,3,0.45,8.8]]}}

for i in range(1, 101):
    if i % 10 == 0:
        print(f"Send {i} requests")
    requests.post(
        url="http://",
        json=body,
        headers=headers,
        verify=False,
        timeout=30
    )

## Optional: Delete AB Model from Seldon Core

```seldon_core_utils.ab_undeployment(name="healthcare_metrics", namespace="default")```